In [1]:
from google.colab import files
uploaded = files.upload()

In [2]:
import pandas as pd

messages=pd.read_csv('/content/SMSSpamCollection', sep='\t',
                           names=["label", "message"])
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Data Pre-Processing

In [3]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [5]:
# Stop word removal and Stemming

corpus = []
for i in range(0, len(messages)):
    #Removing any special characters and converting all text to lowercase and splitting the word 
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    
    #Applying stopwords and Stemming for each word
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [6]:
#Lemmatization
import nltk
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

corpus1 = []
for i in range(0, len(messages)):
    #Removing any special characters and converting all text to lowercase and splitting the word 
    review1 = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review1 = review1.lower()
    review1 = review1.split()
    
    #Applying stopwords and Stemming for each word
    review1 = [lemmatizer.lemmatize(word) for word in review1 if not word in stopwords.words('english')]
    review1 = ' '.join(review1)
    corpus1.append(review1)

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Bag of Words Creation

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X1 = cv.fit_transform(corpus1).toarray()

In [9]:
# Output encoding

y=pd.get_dummies(messages['label'])
y=y.iloc[:,1].values

In [10]:
# Train-Test Split

#Stemmer data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

#Lemmatizer data
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y, test_size = 0.20, random_state = 0)

Using Multinonimal Naive Bayes Model for predicition

In [11]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)
spam_detect_model1 = MultinomialNB().fit(X_train1, y_train1)

In [12]:
#Precition
y_pred=spam_detect_model.predict(X_test)
y_pred1=spam_detect_model.predict(X_test1)

In [13]:
from sklearn.metrics import accuracy_score,classification_report

score=accuracy_score(y_test,y_pred)
print(score)

0.9856502242152466


In [14]:
score1=accuracy_score(y_test1,y_pred1)
print(score1)

0.715695067264574


In [15]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       953
           1       0.96      0.94      0.95       162

    accuracy                           0.99      1115
   macro avg       0.97      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115



In [16]:
print(classification_report(y_pred1,y_test1))

              precision    recall  f1-score   support

           0       0.78      0.88      0.82       850
           1       0.34      0.20      0.25       265

    accuracy                           0.72      1115
   macro avg       0.56      0.54      0.54      1115
weighted avg       0.67      0.72      0.69      1115



Lower accuracy for Lemmatized model in BOW
------WHY?-------

TF-IDF Model

In [17]:
# Creating the TFIDF model
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features=2500)
X = tv.fit_transform(corpus).toarray()

X1 = tv.fit_transform(corpus1).toarray()

In [18]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y, test_size = 0.20, random_state = 0)

In [19]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

spam_detect_model1 = MultinomialNB().fit(X_train1, y_train1)

In [20]:
#Prediction- Stemming
y_pred=spam_detect_model.predict(X_test)
score=accuracy_score(y_test,y_pred)
print(score)
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

0.979372197309417
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       978
           1       0.86      1.00      0.92       137

    accuracy                           0.98      1115
   macro avg       0.93      0.99      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [21]:
#Prediction- Lemmatization
y_pred1=spam_detect_model.predict(X_test1)
score1=accuracy_score(y_test1,y_pred1)
print(score1)

print(classification_report(y_pred1,y_test1))

0.8349775784753363
              precision    recall  f1-score   support

           0       0.95      0.87      0.91      1043
           1       0.15      0.33      0.21        72

    accuracy                           0.83      1115
   macro avg       0.55      0.60      0.56      1115
weighted avg       0.90      0.83      0.86      1115



Accuracy improves for Lemmatizer model in TF IDF

Using Random Forest Model

In [22]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train, y_train)

classifier.fit(X_train1, y_train1)

RandomForestClassifier()

In [23]:
#Prediction- Stemming
y_pred=classifier.predict(X_test)
score=accuracy_score(y_test,y_pred)
print(score)
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

0.852914798206278
              precision    recall  f1-score   support

           0       0.99      0.86      0.92      1097
           1       0.04      0.39      0.08        18

    accuracy                           0.85      1115
   macro avg       0.52      0.62      0.50      1115
weighted avg       0.97      0.85      0.91      1115



In [24]:
#Prediction- Lemmatization
y_pred1=classifier.predict(X_test1)
score1=accuracy_score(y_test1,y_pred1)
print(score1)

print(classification_report(y_pred1,y_test1))

0.9838565022421525
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       973
           1       0.89      1.00      0.94       142

    accuracy                           0.98      1115
   macro avg       0.94      0.99      0.97      1115
weighted avg       0.99      0.98      0.98      1115



Stemming accuracy is lower in Random forest whereas Lemmitization accuracy is great

**Word2vec**

In [25]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
#import gensim.downloader as api
#wv = api.load('word2vec-google-news-300')

In [27]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [28]:
nltk.download('punkt')
words=[]
for sent in corpus1:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Creating a Word2vec model from scratch

In [29]:
import gensim
print(gensim.__version__)

4.3.0


In [30]:
model=gensim.models.Word2Vec(words,window=5,min_count=2)

#first parameter- the words with which you want to train your model
#second param- window size, the number of words to consider consecutively (should be an odd number to have equal left and right context)

In [31]:
model.corpus_count

5564

In [32]:
#Seeing words similar to a given word from the given dataset

model.wv.similar_by_word('even')

[('one', 0.9997972249984741),
 ('get', 0.9997668266296387),
 ('know', 0.9997546672821045),
 ('see', 0.999753475189209),
 ('would', 0.9997448325157166),
 ('ur', 0.9997445940971375),
 ('thing', 0.9997389912605286),
 ('da', 0.9997354745864868),
 ('got', 0.9997305870056152),
 ('go', 0.9997304081916809)]

**Average Word2vec**

In [33]:
import numpy as np
def avg_word2vec(doc):
    # remove out-of-vocabulary words
    #sent = [word for word in doc if word in model.wv.index_to_key]
    #print(sent)
    
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)
                #or [np.zeros(len(model.wv.index_to_key))], axis=0)

In [34]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
from tqdm import tqdm

In [36]:
#apply for the entire sentences
X=[]
for i in tqdm(range(len(words))):
    print("Hello",i)
    X.append(avg_word2vec(words[i]))

  0%|          | 0/5564 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  1%|▏         | 82/5564 [00:00<00:07, 781.96it/s]

Hello 0
Hello 1
Hello 2
Hello 3
Hello 4
Hello 5
Hello 6
Hello 7
Hello 8
Hello 9
Hello 10
Hello 11
Hello 12
Hello 13
Hello 14
Hello 15
Hello 16
Hello 17
Hello 18
Hello 19
Hello 20
Hello 21
Hello 22
Hello 23
Hello 24
Hello 25
Hello 26
Hello 27
Hello 28
Hello 29
Hello 30
Hello 31
Hello 32
Hello 33
Hello 34
Hello 35
Hello 36
Hello 37
Hello 38
Hello 39
Hello 40
Hello 41
Hello 42
Hello 43
Hello 44
Hello 45
Hello 46
Hello 47
Hello 48
Hello 49
Hello 50
Hello 51
Hello 52
Hello 53
Hello 54
Hello 55
Hello 56
Hello 57
Hello 58
Hello 59
Hello 60
Hello 61
Hello 62
Hello 63
Hello 64
Hello 65
Hello 66
Hello 67
Hello 68
Hello 69
Hello 70
Hello 71
Hello 72
Hello 73
Hello 74
Hello 75
Hello 76
Hello 77
Hello 78
Hello 79
Hello 80
Hello 81
Hello 82
Hello 83
Hello 84
Hello 85
Hello 86
Hello 87
Hello 88
Hello 89
Hello 90
Hello 91
Hello 92
Hello 93
Hello 94
Hello 95
Hello 96
Hello 97
Hello 98
Hello 99
Hello 100
Hello 101
Hello 102
Hello 103
Hello 104
Hello 105
Hello 106
Hello 107
Hello 108
Hello 109
Hello 110


  4%|▍         | 249/5564 [00:00<00:06, 797.24it/s]

Hello 160
Hello 161
Hello 162
Hello 163
Hello 164
Hello 165
Hello 166
Hello 167
Hello 168
Hello 169
Hello 170
Hello 171
Hello 172
Hello 173
Hello 174
Hello 175
Hello 176
Hello 177
Hello 178
Hello 179
Hello 180
Hello 181
Hello 182
Hello 183
Hello 184
Hello 185
Hello 186
Hello 187
Hello 188
Hello 189
Hello 190
Hello 191
Hello 192
Hello 193
Hello 194
Hello 195
Hello 196
Hello 197
Hello 198
Hello 199
Hello 200
Hello 201
Hello 202
Hello 203
Hello 204
Hello 205
Hello 206
Hello 207
Hello 208
Hello 209
Hello 210
Hello 211
Hello 212
Hello 213
Hello 214
Hello 215
Hello 216
Hello 217
Hello 218
Hello 219
Hello 220
Hello 221
Hello 222
Hello 223
Hello 224
Hello 225
Hello 226
Hello 227
Hello 228
Hello 229
Hello 230
Hello 231
Hello 232
Hello 233
Hello 234
Hello 235
Hello 236
Hello 237
Hello 238
Hello 239
Hello 240
Hello 241
Hello 242
Hello 243
Hello 244
Hello 245
Hello 246
Hello 247
Hello 248
Hello 249
Hello 250
Hello 251
Hello 252
Hello 253
Hello 254
Hello 255
Hello 256
Hello 257
Hello 258
Hello 259


  7%|▋         | 412/5564 [00:00<00:07, 723.65it/s]

...

Hello 5193
Hello 5194
Hello 5195
Hello 5196
Hello 5197
Hello 5198
Hello 5199
Hello 5200
Hello 5201
Hello 5202
Hello 5203
Hello 5204
Hello 5205
Hello 5206
Hello 5207
Hello 5208
Hello 5209
Hello 5210
Hello 5211
Hello 5212
Hello 5213
Hello 5214
Hello 5215
Hello 5216
Hello 5217
Hello 5218
Hello 5219
Hello 5220
Hello 5221
Hello 5222
Hello 5223
Hello 5224
Hello 5225
Hello 5226
Hello 5227
Hello 5228
Hello 5229
Hello 5230
Hello 5231
Hello 5232
Hello 5233
Hello 5234
Hello 5235
Hello 5236
Hello 5237
Hello 5238
Hello 5239
Hello 5240
Hello 5241
Hello 5242
Hello 5243
Hello 5244
Hello 5245
Hello 5246
Hello 5247
Hello 5248
Hello 5249
Hello 5250
Hello 5251
Hello 5252
Hello 5253
Hello 5254
Hello 5255
Hello 5256
Hello 5257
Hello 5258
Hello 5259
Hello 5260
Hello 5261
Hello 5262
Hello 5263
Hello 5264
Hello 5265
Hello 5266
Hello 5267
Hello 5268
Hello 5269
Hello 5270
Hello 5271
Hello 5272
Hello 5273
Hello 5274
Hello 5275
Hello 5276
Hello 5277
Hello 5278
Hello 5279
Hello 5280
Hello 5281
Hello 5282
Hello 5283

 99%|█████████▉| 5533/5564 [00:07<00:00, 811.07it/s]

Hello 5365
Hello 5366
Hello 5367
Hello 5368
Hello 5369
Hello 5370
Hello 5371
Hello 5372
Hello 5373
Hello 5374
Hello 5375
Hello 5376
Hello 5377
Hello 5378
Hello 5379
Hello 5380
Hello 5381
Hello 5382
Hello 5383
Hello 5384
Hello 5385
Hello 5386
Hello 5387
Hello 5388
Hello 5389
Hello 5390
Hello 5391
Hello 5392
Hello 5393
Hello 5394
Hello 5395
Hello 5396
Hello 5397
Hello 5398
Hello 5399
Hello 5400
Hello 5401
Hello 5402
Hello 5403
Hello 5404
Hello 5405
Hello 5406
Hello 5407
Hello 5408
Hello 5409
Hello 5410
Hello 5411
Hello 5412
Hello 5413
Hello 5414
Hello 5415
Hello 5416
Hello 5417
Hello 5418
Hello 5419
Hello 5420
Hello 5421
Hello 5422
Hello 5423
Hello 5424
Hello 5425
Hello 5426
Hello 5427
Hello 5428
Hello 5429
Hello 5430
Hello 5431
Hello 5432
Hello 5433
Hello 5434
Hello 5435
Hello 5436
Hello 5437
Hello 5438
Hello 5439
Hello 5440
Hello 5441
Hello 5442
Hello 5443
Hello 5444
Hello 5445
Hello 5446
Hello 5447
Hello 5448
Hello 5449
Hello 5450
Hello 5451
Hello 5452
Hello 5453
Hello 5454
Hello 5455

100%|██████████| 5564/5564 [00:07<00:00, 747.43it/s]

 5544
Hello 5545
Hello 5546
Hello 5547
Hello 5548
Hello 5549
Hello 5550
Hello 5551
Hello 5552
Hello 5553
Hello 5554
Hello 5555
Hello 5556
Hello 5557
Hello 5558
Hello 5559
Hello 5560
Hello 5561
Hello 5562
Hello 5563
